In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras

In [2]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

In [3]:
image_size=32
def preprocess (image_path, image_height=image_size, image_width=image_size):
  img_load = [load_img( img , target_size = (image_height, image_width)) for img in image_path]
  data_array = np.array([ img_to_array(img) for img in img_load ])
  # output = preprocess_input(data_array)
  return data_array

In [4]:
data_main_df = pd.read_csv('/content/drive/My Drive/Kaggle practice files/Corona hack chest x ray/datasets_562468_1022626_Chest_xray_Corona_Metadata (1).csv')
data_summary_df = pd.read_csv('/content/drive/My Drive/Kaggle practice files/Corona hack chest x ray/_Corona_dataset_Summary.csv')

data_df = pd.concat([data_main_df,data_summary_df])

data_df


,Unnamed: 0,X_ray_image_name,Label,Dataset_type,Label_2_Virus_category,Label_1_Virus_category,Image_Count
0,0,IM-0128-0001.jpeg,Normal,TRAIN,NaN,NaN,NaN
1,1,IM-0127-0001.jpeg,Normal,TRAIN,NaN,NaN,NaN
2,2,IM-0125-0001.jpeg,Normal,TRAIN,NaN,NaN,NaN
3,3,IM-0122-0001.jpeg,Normal,TRAIN,NaN,NaN,NaN
4,4,IM-0119-0001.jpeg,Normal,TRAIN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2,2,NaN,Pnemonia,NaN,NaN,Virus,1493.0
3,3,NaN,Pnemonia,NaN,COVID-19,Virus,58.0
4,4,NaN,Pnemonia,NaN,SARS,Virus,4.0
5,5,NaN,Pnemonia,NaN,NaN,bacteria,2772.0


## SPLITTING TRAIN AND TEST DATA

In [ ]:
data_train_df = data_df[['X_ray_image_name','Label']].where(data_df.Dataset_type == 'TRAIN')
data_train_df= data_train_df.dropna()
print(data_train_df.shape)
# data_train_df

data_test_df = data_df[['X_ray_image_name','Label']].where(data_df.Dataset_type == 'TEST')
data_test_df= data_test_df.dropna()
print(data_test_df.shape)


(5286, 2)
(624, 2)


In [ ]:
 data_train_df.Label.unique()

array(['Normal', 'Pnemonia'], dtype=object)

In [ ]:
from os.path import join
data_path_train = '/content/drive/My Drive/Kaggle practice files/Corona hack chest x ray/Coronahack-Chest-XRay-Dataset/train'
img_paths = [join(data_path_train, filename) for filename in data_train_df['X_ray_image_name']]

In [ ]:
data_train = preprocess(img_paths)

In [ ]:
X = data_train

In [ ]:
# Normalize data and bring to range 0-1
X = X.astype('float32')
X = X/255
X.shape

(5286, 32, 32, 3)

In [ ]:
# Converting class vectors to binary class metrics
# Our class are 1 and 2
num_classe = 2
data_train_df = pd.DataFrame(data_train_df.Label.replace({'Normal':0, 'Pnemonia':1}), columns= ['Label'])  #Mapping Normal:1 and Pnemonia: 2
Y = keras.utils.to_categorical(data_train_df.Label, num_classe)

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from sklearn.model_selection import train_test_split

In [ ]:
model = Sequential()
model.add(Conv2D(50, activation='relu', kernel_size=(3,3), input_shape=(32,32,3)))
model.add(Conv2D(50, kernel_size=(3,3), activation='relu'))
model.add(Flatten())  # converts 2D to 1D
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classe, activation='softmax'))  # output layer

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, test_size=0.2 )

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test,y_test), batch_size=20, epochs=4,shuffle=True)

Epoch 1/4
212/212 [==============================] - 32s 150ms/step - loss: 0.2498 - accuracy: 0.8957 - val_loss: 0.2056 - val_accuracy: 0.9187
Epoch 2/4
212/212 [==============================] - 28s 130ms/step - loss: 0.1268 - accuracy: 0.9539 - val_loss: 0.1205 - val_accuracy: 0.9537
Epoch 3/4
212/212 [==============================] - 28s 130ms/step - loss: 0.1121 - accuracy: 0.9596 - val_loss: 0.1774 - val_accuracy: 0.9414
Epoch 4/4
212/212 [==============================] - 28s 132ms/step - loss: 0.0924 - accuracy: 0.9607 - val_loss: 0.1153 - val_accuracy: 0.9603


## TEST DATA

In [ ]:
from os.path import join
data_path_test = '/content/drive/My Drive/Kaggle practice files/Corona hack chest x ray/Coronahack-Chest-XRay-Dataset/test'
img_paths_test = [join(data_path_test, filename) for filename in data_test_df['X_ray_image_name']]

In [ ]:
data_test = preprocess(img_paths_test)

In [ ]:
x_valid = data_test
x_valid = x_valid.astype('float32')
x_valid = x_valid/255

In [ ]:
data_test_df = pd.DataFrame(data_test_df.Label.replace({'Normal':0, 'Pnemonia':1}), columns= ['Label'])  #Mapping Normal:1 and Pnemonia: 2
y_valid = keras.utils.to_categorical(data_test_df.Label, num_classe)

## PREDICTING 

In [ ]:
preds = model.predict(x_valid)

In [ ]:
# Generate arg maxes for predictions
classes = np.argmax(preds, axis = 1)
print(classes)

[1 1 0 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 0 1 0 0
 1 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 1 0 0 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 0 1 0 1 1 1 1 1 0 1
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 0 1
 1 1 1 0 0 1 1 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0
 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
mean_absolute_error(classes, data_test_df )

0.2644230769230769

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
precision_recall_fscore_support(data_test_df, classes, average='weighted')

(0.7965615280514732, 0.7355769230769231, 0.6909863181049186, None)